In [ ]:
import pandas as pd
import time
import os
from datetime import datetime
from collections import defaultdict

!pip install polars==0.18.2
import polars as pl

In [ ]:
%cd drive/My\ Drive/performance_prediction

In [10]:
# convert kaggle data to parquet
pd.read_csv("data/train.csv").to_parquet("data/train.parquet")

In [ ]:
# parse raw data

files1 = [
    'data/JOWILDER_20200401_to_20200430/JOWILDER_20200401_to_20200430_7fbb180_events',
    "data/JOWILDER_20200801_to_20200831/JOWILDER_20200801_to_20200831_7fbb180_events",
    "data/JOWILDER_20200901_to_20200930/JOWILDER_20200901_to_20200930_7fbb180_events",
]

files2 = [
    'data/JOWILDER_20211101_to_20211130/JOWILDER_20211101_to_20211130_42454c6_events',
    'data/JOWILDER_20220701_to_20220731/JOWILDER_20220701_to_20220731_e3039cf_events',
    'data/JOWILDER_20220801_to_20220831/JOWILDER_20220801_to_20220831_e3039cf_events',
    'data/JOWILDER_20220601_to_20220630/JOWILDER_20220601_to_20220630_e3039cf_events',
    'data/JOWILDER_20210701_to_20210731/JOWILDER_20210701_to_20210731_42454c6_events',
    'data/JOWILDER_20210801_to_20210831/JOWILDER_20210801_to_20210831_42454c6_events',
    'data/JOWILDER_20210901_to_20210930/JOWILDER_20210901_to_20210930_42454c6_events',
    'data/JOWILDER_20211201_to_20211231/JOWILDER_20211201_to_20211231_42454c6_events',
    'data/JOWILDER_20211001_to_20211031/JOWILDER_20211001_to_20211031_42454c6_events',
    'data/JOWILDER_20210601_to_20210630/JOWILDER_20210601_to_20210630_42454c6_events',
    'data/JOWILDER_20210101_to_20210131/JOWILDER_20210101_to_20210131_42454c6_events',
    'data/JOWILDER_20210201_to_20210228/JOWILDER_20210201_to_20210228_42454c6_events',
    "data/JOWILDER_20221001_to_20221031/JOWILDER_20221001_to_20221031_40e77cd_events",
    "data/JOWILDER_20221101_to_20221130/JOWILDER_20221101_to_20221130_40e77cd_events",
    "data/JOWILDER_20191101_to_20191130/JOWILDER_20191101_to_20191130_10bbaaf_events",
    "data/JOWILDER_20220301_to_20220331/JOWILDER_20220301_to_20220331_b751607_events",
    "data/JOWILDER_20191201_to_20191231/JOWILDER_20191201_to_20191231_10bbaaf_events",
    "data/JOWILDER_20200101_to_20200131/JOWILDER_20200101_to_20200131_10bbaaf_events",
    "data/JOWILDER_20220501_to_20220531/JOWILDER_20220501_to_20220531_93eaf7d_events",
    "data/JOWILDER_20200201_to_20200229/JOWILDER_20200201_to_20200229_10bbaaf_events",
    "data/JOWILDER_20200301_to_20200331/JOWILDER_20200301_to_20200331_10bbaaf_events",
    "data/JOWILDER_20200501_to_20200531/JOWILDER_20200501_to_20200531_10bbaaf_events",
    "data/JOWILDER_20200601_to_20200630/JOWILDER_20200601_to_20200630_10bbaaf_events",
    "data/JOWILDER_20200701_to_20200731/JOWILDER_20200701_to_20200731_10bbaaf_events",
    "data/JOWILDER_20220101_to_20220131/JOWILDER_20220101_to_20220131_1df5a3a_events",
    "data/JOWILDER_20220201_to_20220228/JOWILDER_20220201_to_20220228_dbfe71d_events",
    "data/JOWILDER_20220401_to_20220430/JOWILDER_20220401_to_20220430_93eaf7d_events",
    "data/JOWILDER_20220901_to_20220930/JOWILDER_20220901_to_20220930_6228b2e_events",
]

def convert_to_timestamp(date_string, formats):
    """
    Converts a date string in some of the formats in the "formats" list to milliseconds
    """
    for date_format in formats:
        try:
            return datetime.strptime(date_string, date_format).timestamp() * 1000
        except ValueError:
            pass
    raise ValueError(f"No valid format found for {date_string}")

formats = ["%Y-%m-%d %H:%M:%S.%f", "%Y-%m-%d %H:%M:%S", "%Y-%m-%dT%H:%M:%S.%f", "%Y-%m-%dT%H:%M:%S"]


for file in (files1 + files2):
    print(file)

    cs = ["session_id", "timestamp", "event_data", "index"]

    # the raw data in files1 has no index,
    # and the time is split in two parts, client_time and client_time_ms
    if file in files1:
        try:
            data = pl.read_csv(file + ".tsv", separator="\t")
        except:
            data = pl.read_csv(file + ".tsv", separator="\t", dtypes={"session_n": str})
        data = data.with_columns(pl.arange(0, pl.col("app_id_fast").count()).over(["session_id"]).alias("index"))
        data = data.with_columns((pl.col("client_time") + "." + pl.col("client_time_ms").cast(pl.Utf8)).alias("timestamp")).rename({"event_data_complex": "event_data"})
        data = data.select(cs)
    else:
        data = pl.read_csv(file + ".tsv", separator="\t").select(cs)

    # create the level column,
    # for doing some initial filtering of sessions, to speed up the parsing
    data = data.with_columns(pl.col("event_data").str.extract('["\']level["\']: (\d+)', 1).cast(pl.Int32).alias("level"))
    data = data.fill_null(-1)
    data = data.filter(((pl.col("level").max() > 12) & ((pl.col("level") == 0) & (pl.col("index") > 1)).any() & (pl.col("level") < 23)).over("session_id"))


    cols = pl.read_parquet("data/train.parquet", n_rows=1).columns
    data = data.rows(named=True)
    out_data = {c: [] for c in cols}
    labels = {}
    fails = []

    # iterate over the rows in the raw data and parse each row
    for i, r in enumerate(data):

        e = r["event_data"]
        e = e.replace("true", "1")
        e = e.replace("false", "0")
        e = e.replace("null", "None")
        e = eval(e)

        typ = e["type"]
        subtyp = e["subtype"]

        # skip all rows like these, that are not present in the kaggle data
        if typ in ["quiz", "endgame", "quizquestion"]:
            continue

        if typ == "startgame":
            hq = e["hq"]
            music = e["music"]
            fullscreen = e["fullscreen"]
            continue


        # parse labels
        sid = r["session_id"]
        if sid not in labels:
            labels[sid] = [1] * 18
        if "cur_cmd_fqid" in e:
            ans = e["cur_cmd_fqid"]
            if ans in [
                "tunic.capitol_0.hall.boss.chap1_finale_slipfirst_0_fail",
                "tunic.capitol_0.hall.boss.chap1_finale_plaquefirst_0_fail"
            ]:
                labels[sid][2] = 0
            for q in range(3):
                chap, hall = 1, 0
                if ans == f"tunic.capitol_{hall}.hall.boss.chap{chap}_finale_{q}_fail":
                    labels[sid][q] = 0
            for q in range(6):
                chap, hall = 2, 1
                if ans == f"tunic.capitol_{hall}.hall.boss.chap{chap}_finale_{q}_fail":
                    labels[sid][q + 3] = 0
            if ans == f"tunic.capitol_1.hall.gramps.chap2_teddy_finale_0_fail":
                labels[sid][9] = 0
            if ans == f"tunic.capitol_1.hall.boss.chap2_teddy_finale_1_fail":
                labels[sid][10] = 0
            if ans == f"tunic.capitol_1.hall.gramps.chap2_teddy_finale_2_fail":
                labels[sid][11] = 0
            if ans == f"tunic.capitol_1.hall.wells.chap2_teddy_finale_3_fail":
                labels[sid][12] = 0
            for q in range(5):
                chap, hall = 4, 2
                if ans == f"tunic.capitol_{hall}.hall.boss.chap{chap}_finale_{q}_fail":
                    labels[sid][q + 13] = 0


        fqid = e["fqid"]
        if "finale" in str(fqid) and subtyp in ["wildcard", "notebook"]:
            continue
        if fqid == "credits":
            continue
        if e["room_fqid"] in ["tunic.capitol_0.hall", "tunic.capitol_1.hall", "tunic.capitol_2.hall"] and subtyp == "notebook":
            continue

        t = r["timestamp"]
        et = convert_to_timestamp(t, formats)

        # Skipping rows after the checkpoint event most of the time.
        # In the kaggle data though there are rows after the checkpoint event,
        # if the elapsed_time is decreasing (reversed). Keeping those here to
        if len(out_data["event_name"]) > 0 and "checkpoint" in out_data["event_name"] and \
         e["level"] in [4, 12, 22] and out_data["session_id"][cpt_ix] == sid and \
         out_data["level"][cpt_ix] == e["level"] and out_data["elapsed_time"][cpt_ix] < et:
                continue

        if len(out_data["session_id"]) > 0 and out_data["session_id"][-1] != sid and sid in out_data["session_id"]:
            continue

        if typ == "click":
            event_name = subtyp + "_" + typ
        else:
            event_name = typ

        if typ == "checkpoint":
            cpt_ix = len(out_data["event_name"])


        if typ == "hover":
            event_name = subtyp + "_" + typ
            # just for simplicity, dropping the hover rows before training anyway
            out_data["hover_duration"].append(1)
        else:
            out_data["hover_duration"].append(None)

        level = e["level"]
        if level < 5:
            lg = "0-4"
        elif 5 <= level <= 12:
            lg = "5-12"
        else:
            lg = "13-22"

        # append the parsed data
        out_data["level_group"].append(lg)
        out_data["index"].append(r["index"])
        out_data["event_name"].append(event_name)
        out_data["session_id"].append(sid)
        out_data["name"].append(e["name"])
        out_data["text"].append(None if "text" not in e else e["text"])
        out_data["page"].append(None if "page" not in e else e["page"])
        out_data["text_fqid"].append(None if "text_fqid" not in e else e["text_fqid"])
        out_data["fqid"].append(fqid if fqid else None)
        out_data["level"].append(level)
        out_data["room_fqid"].append(e["room_fqid"])
        out_data["room_coor_x"].append(None if "room_coor" not in e else e["room_coor"][0])
        out_data["room_coor_y"].append(None if "room_coor" not in e else e["room_coor"][1])
        out_data["screen_coor_x"].append(None if "screen_coor" not in e else e["screen_coor"][0])
        out_data["screen_coor_y"].append(None if "screen_coor" not in e else e["screen_coor"][1])
        out_data["hq"].append(hq)
        out_data["music"].append(music)
        out_data["fullscreen"].append(fullscreen)
        out_data["elapsed_time"].append(et)



    d = {k: v for k, v in out_data.items() if len(v) > 0}
    df = pl.DataFrame(d)

    df = df.with_columns(pl.col("elapsed_time") - pl.col("elapsed_time").min().over("session_id"))
    df = df.filter(~(pl.col("hover_duration").is_null() & pl.col("room_coor_x").is_null() & (pl.col("event_name") != "checkpoint")))

    labels = {k: v for k, v in labels.items() if k in df["session_id"].unique()}
    sids = {"session_id": [k for k in labels.keys()]}
    qs = {f"q{i}": [v[i] for v in labels.values()] for i in range(18)}
    labels = pl.DataFrame(sids | qs)

    df.write_parquet(file + "_train.parquet")
    labels.write_parquet(file + "_labels.parquet")

In [ ]:
# concatenate the parsed data from the files

cs_cols = [
    pl.col("screen_coor_x").cast(pl.Float64),
    pl.col("screen_coor_y").cast(pl.Float64),
]
ds = [pl.read_parquet(f + "_train.parquet").with_columns(cs_cols) for f in files1 + files2]
df = pl.concat(ds)
print(df.shape)
df.write_parquet("data/raw_train.parquet")

ds = [pl.read_parquet(f + "_labels.parquet") for f in files1 + files2]
df = pl.concat(ds)
print(df.shape)
df.write_parquet("data/raw_labels.parquet")